In [56]:
import os
import pandas as pd
from functools import reduce
import numpy as np
from datetime import date
import tensorflow as tf
from keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pmdarima as pm


In [57]:
df=pd.read_csv("C:/Users/91966/OneDrive/Desktop/capstone/telangana/weather/daily_weather_data.csv")
df=df[['district','odate','rainfall','temp_min','temp_max']].dropna()

In [58]:
df1=df.copy(deep=True)
df2=df.copy(deep=True)
df3=df.copy(deep=True)
df1['odate'] = pd.to_datetime(df1['odate'], format='%d-%m-%Y', errors='coerce')
df1.dropna(inplace=True)
df2['odate'] = pd.to_datetime(df2['odate'], format='%Y-%m-%d', errors='coerce')
df2.dropna(inplace=True)
df3['odate'] = pd.to_datetime(df3['odate'], format='%d-%b-%y', errors='coerce')
df3.dropna(inplace=True)

In [59]:
df_final=pd.concat([df1,df2,df3],axis=0).sort_index()
df_final=df_final.sort_values(by='odate')

In [60]:
df_comps={}
options=["Adilabad","Nizamabad","Warangal","Karimnagar","Khammam"]
for i in options:
  df_comps[i]=df_final[df_final["district"]==i]

In [61]:
df_comps['Adilabad']=df_comps['Adilabad'][['odate','temp_max','temp_min']]

In [62]:
for i in df_comps:
    df_comps[i]=df_comps[i].groupby(by=["odate"]).mean()


C:\Users\91966\AppData\Local\Temp\ipykernel_19552\366426764.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_comps[i]=df_comps[i].groupby(by=["odate"]).mean()
C:\Users\91966\AppData\Local\Temp\ipykernel_19552\366426764.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_comps[i]=df_comps[i].groupby(by=["odate"]).mean()
C:\Users\91966\AppData\Local\Temp\ipykernel_19552\366426764.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid

In [63]:
df_comps['Adilabad']

,temp_max,temp_min
odate,,
2018-01-01,30.436842,11.200000
2018-01-02,31.173684,11.810526
2018-01-03,31.121053,11.626316
2018-01-04,30.300000,11.200000
2018-01-05,29.410526,10.700000
...,...,...
2022-09-26,33.972222,23.544444
2022-09-27,33.350000,24.683333
2022-09-28,33.894444,23.661111


In [64]:
df=df_comps['Adilabad']

df["avg_temp"] = ((df["temp_max"] + df["temp_min"])/2)

In [65]:
holt_winter = sm.tsa.ExponentialSmoothing(
    df['temp_max'], seasonal='add', seasonal_periods=365)
x = holt_winter.fit()
x.params

c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


{'smoothing_level': 0.7886079908393024,
 'smoothing_trend': nan,
 'smoothing_seasonal': 6.7756585193467504e-09,
 'damping_trend': nan,
 'initial_level': 36.38709664346721,
 'initial_trend': nan,
 'initial_seasons': array([-5.97982845, -5.17923061, -5.12015106, -5.00904019, -4.76510899,
        -3.82804973, -3.72887276, -4.91137641, -5.8917726 , -4.95064257,
        -5.21954556, -4.70517285, -4.46652583, -3.59241116, -3.40429216,
        -5.32601615, -4.03399701, -2.95407572, -3.38995581, -3.73236269,
        -2.38711093, -2.69874656, -2.0510423 , -2.69031822, -3.38365258,
        -4.25958126, -5.10462182, -5.05046876, -4.83293288, -4.92759728,
        -4.46488175, -1.90600285, -1.79498776, -1.07561535, -0.72975935,
        -1.59859804, -0.75134598, -0.9942769 , -3.57687039, -2.23460802,
        -2.6989107 , -2.31072834, -0.75417313, -0.78136919, -0.10702882,
        -0.50070392, -0.28239105, -0.3342797 , -0.72672474, -0.48685979,
        -0.90549739, -0.08339466,  1.08671541,  2.009889

In [66]:
holt_winter1 = sm.tsa.ExponentialSmoothing(
    df['avg_temp'], seasonal='add', seasonal_periods=365)
y = holt_winter1.fit()
y.params

c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\holtwinters\model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


{'smoothing_level': 0.9162903102779868,
 'smoothing_trend': nan,
 'smoothing_seasonal': 1.0289697253133411e-09,
 'damping_trend': nan,
 'initial_level': 27.941709822792326,
 'initial_trend': nan,
 'initial_seasons': array([-7.11117703, -6.58604042, -6.65663965, -6.63412762, -6.78116063,
        -5.24776375, -4.44123702, -5.09235274, -6.74259954, -6.59089733,
        -6.39855945, -5.55267418, -5.08075962, -4.40849141, -4.41408718,
        -5.97687909, -5.54318022, -4.89864688, -5.35800749, -5.6940855 ,
        -4.49123801, -4.60621529, -3.67670095, -4.03218928, -4.76831308,
        -5.69284819, -5.69774554, -6.28768683, -6.68706968, -7.60713374,
        -6.97682993, -5.11771307, -4.95533049, -4.38125508, -3.79205687,
        -4.78402733, -4.03268423, -2.91015332, -5.22931704, -4.80447165,
        -5.22702744, -4.59300768, -3.6813469 , -3.69899146, -3.1479309 ,
        -2.61089244, -2.86468575, -3.2539084 , -3.35233668, -2.66719599,
        -2.22178176, -1.89872425, -1.08768751, -0.66932

KeyError: 'odate'

In [68]:
df['temp_max_pred'] = list(x.predict(1, len(df)))
# df[['aqi', 'predict_holt']].plot()

df1=pd.DataFrame(x.forecast(365))
df1.columns=['temp_max_pred']
df1

    

c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


,temp_max_pred
1733,33.076626
1734,32.864294
1735,32.397880
1736,33.227735
1737,32.964409
...,...
2093,31.933792
2094,31.837247
2095,31.782182
2096,31.898388


In [69]:
def prob(mae_tempmax,mae_tempavg,temp_max,temp_avg):
    final_temp_max=temp_max-mae_tempmax
    final_temp_avg=temp_avg+mae_tempavg
    prob=min((final_temp_max-final_temp_avg)/4.5,0.99)*100
    prob_severe=min((final_temp_max-final_temp_avg)/6.4,0.99)*100
    return prob,prob_severe

In [70]:
df['temp_avg_pred'] = list(y.predict(1, len(df)))
# df[['aqi', 'predict_holt']].plot()
df2=pd.DataFrame(y.forecast(365))
df2.columns=['temp_avg_pred']
df_final=pd.concat([df1,df2],axis=1)
df_final

c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\91966\anaconda3\envs\tf-gpu\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


,temp_max_pred,temp_avg_pred
1733,33.076626,28.287190
1734,32.864294,28.013930
1735,32.397880,27.800154
1736,33.227735,27.929832
1737,32.964409,27.903267
...,...,...
2093,31.933792,28.049044
2094,31.837247,27.981451
2095,31.782182,27.563879
2096,31.898388,27.674453


In [77]:
df_final['Normal_heat_wave_prediction']=df_final.apply(lambda x:prob(0.4,0.2,x['temp_max_pred'],x['temp_avg_pred'])[0],axis=1)
df_final['Severe_heat_wave_prediction']=df_final.apply(lambda x:prob(0.4,0.2,x['temp_max_pred'],x['temp_avg_pred'])[1],axis=1)

In [78]:
df_final

,temp_max_pred,temp_avg_pred,prediction,Normal_heat_wave_prediction,Severe_heat_wave_prediction
1733,33.076626,28.287190,"(93.09857221648186, 65.4599335897138)",93.098572,65.459934
1734,32.864294,28.013930,"(94.45253449048181, 66.41193831362003)",94.452534,66.411938
1735,32.397880,27.800154,"(88.83834188339044, 62.46445913675891)",88.838342,62.464459
1736,33.227735,27.929832,"(99.0, 73.40474166484584)",99.000000,73.404742
1737,32.964409,27.903267,"(99.0, 69.70535085647501)",99.000000,69.705351
...,...,...,...,...,...
2093,31.933792,28.049044,"(72.99440629760636, 51.32419192800447)",72.994406,51.324192
2094,31.837247,27.981451,"(72.35102071586839, 50.87181144084496)",72.351021,50.871811
2095,31.782182,27.563879,"(80.40672296395073, 56.535977084027856)",80.406723,56.535977
2096,31.898388,27.674453,"(80.53189741196047, 56.62399036778471)",80.531897,56.623990
